In [2]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification

In [15]:
num_col, num_row = 100, 1000

In [16]:
X, y = make_classification(
    n_samples=num_row,
    n_features=num_col,
    n_informative=1,
    n_redundant=0,
    n_clusters_per_class=1,
    random_state=720,
    weights=(0.9,0.1)
)

In [17]:
cols = list()
for i in range(1, num_col+1):
    cols.append(f'column{i}')

In [18]:
res_df = pd.DataFrame(data=X, columns=cols)
res_df['target'] = y

In [19]:
import random
from datetime import datetime, timedelta

In [20]:
def generate_random_offsets_in_month(year, month, num_offsets):
    # Calculate the number of days in the specified month
    days_in_month = (datetime(year, month % 12 + 1, 1) - timedelta(days=1)).day

    # Set the start and end datetime for the specified month
    start_datetime = datetime(year, month, 1, 0, 0, 0)
    end_datetime = datetime(year, month, days_in_month, 23, 59, 59)

    # Calculate the time difference in seconds
    time_diff_seconds = (end_datetime - start_datetime).total_seconds()

    # Generate num_offsets random offsets in seconds within the range
    random_offsets = [random.randint(0, int(time_diff_seconds)) for _ in range(num_offsets)]

    return random_offsets

def random_datetimes_in_month(year, month, num_datetimes):
    random_offsets = generate_random_offsets_in_month(year, month, num_datetimes)

    # Get the start datetime for the specified month
    start_datetime = datetime(year, month, 1, 0, 0, 0)

    # Generate random datetimes using the precomputed random offsets
    random_datetimes = [start_datetime + timedelta(seconds=offset) for offset in random_offsets]

    return random_datetimes

In [21]:
# Example usage for January (month=1) of the year 2023 with almost a million random datetimes
year = 2023
month = 1
random_datetimes = random_datetimes_in_month(year, month, num_row)

In [22]:
res_df['date_time'] = random_datetimes

In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark AWS Credentials Example") \
    .getOrCreate()

In [24]:
df = spark.createDataFrame(res_df)

In [25]:
df.write.mode('append').parquet("s3a://test-ml-storage/ml-test/aizen_de_test_1.parquet")

23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/07/25 13:28:35 WARN MemoryManager: Total allocation exceeds 95.

In [32]:
spark.stop()